In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
labelencoder=preprocessing.LabelEncoder()
data = pd.read_csv("insurance.csv")
data.drop_duplicates(inplace=True)
data["smoker"] = labelencoder.fit_transform(data["smoker"])
data["sex"] = labelencoder.fit_transform(data["sex"])
data["region"] = labelencoder.fit_transform(data["region"])
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [4]:
x = data[["age", "sex", "bmi", "children", "smoker", "region"]]
y = data["charges"]

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_x_train = scaler.fit_transform(x_train)
scaled_x_test = scaler.fit_transform(x_test)

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
def modelresults(predictions):
    print("Mean absolute error on model is {}".format(mean_absolute_error(y_test, predictions)))
    print("Root mean squared error on model is {}".format(np.sqrt(mean_squared_error(y_test, predictions))))

In [24]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(scaled_x_train, y_train)

LinearRegression()

In [25]:
predslr = lr.predict(scaled_x_test)
modelresults(predslr)

Mean absolute error on model is 4298.783917296326
Root mean squared error on model is 6149.7935303232


In [27]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
svrmodel = SVR()
param_gridsvr = {'C':[0.001, 0.01, 0.1, 0.5, 1], 'kernel':['linear', 'rbf', 'poly'], 'gamma':['scale', 'auto'], 'degree':[2,3,4,5]}
gridsvr = GridSearchCV(svrmodel, param_gridsvr)
gridsvr.fit(scaled_x_train, y_train)
print("Best parameters for model are{}".format(gridsvr.best_params_))

Best parameters for model are{'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}


In [30]:
predsgridsvr = gridsvr.predict(scaled_x_test)
modelresults(predsgridsvr)

Mean absolute error on model is 7606.311282002542
Root mean squared error on model is 11970.997272831606


In [32]:
from sklearn.ensemble import RandomForestRegressor
rfrmodel = RandomForestRegressor()
param_gridrfr = {'bootstrap':[True], 'max_depth':[5,10,15], 'max_features':['auto', 'log2'], 'n_estimators':[2,3,4,5,6,7,8,9,10]}
gridrfr = GridSearchCV(rfrmodel, param_gridrfr)
gridrfr.fit(scaled_x_train, y_train)

C:\Users\nawaz\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
135 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nawaz\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nawaz\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\nawaz\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_const

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True], 'max_depth': [5, 10, 15],
                         'max_features': ['auto', 'log2'],
                         'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10]})

In [33]:
predsgridrfr = gridrfr.predict(scaled_x_test)
modelresults(predsgridrfr)

Mean absolute error on model is 3165.0410916136852
Root mean squared error on model is 5070.627484914182


In [35]:
columniterate = 1
for index in x.columns:
    mean = data[index].mean()
    print("The mean of the column {} is {}".format(columniterate, mean))
    columniterate += 1

The mean of the column 1 is 39.222139117427076
The mean of the column 2 is 0.5048616305160808
The mean of the column 3 is 30.66345175766642
The mean of the column 4 is 1.0957367240089753
The mean of the column 5 is 0.2049364248317128
The mean of the column 6 is 1.5160807778608825


In [36]:
newcustomer = np.array([39,0,30,1,0,1])

In [37]:
gridrfr.predict(newcustomer.reshape(1,-1))

array([11660.94451087])

In [38]:
print("The insurance cost of new customer is {}".format(gridrfr.predict(newcustomer.reshape(1,-1))[0])) 

The insurance cost of new customer is 11660.944510868421
